# Discrete analysis: preprocessing

This notebook is for the longer running compute tasks for the discrete analysis, the results of which are evaluated in the related notebooks.

In [ ]:
# config cell
from pathlib import Path
from dask.distributed import Client

# for reloading the baeda module which is actively developing
import baeda
import importlib

importlib.reload(baeda)
from baeda import projected_coords, tmp_era5_discrete_idx_fp

var_ids = ["tasmax", "pr"]
coords = projected_coords

tmp_dir = Path("/center1/CMIP6/kmredilla/downscaling/eda")
era5_dir = Path("/center1/CMIP6/kmredilla/cmip6_4km_downscaling/era5_zarr")

Compute the indicators for the ERA5 data

In [ ]:
# First, compute the indicators for ERA5 data for packaging with GCM results:

# load ERA5

client = Client()

# we are loading both precip and temp to choose some locations based on extremes in these variables
era5_stores = {
    "t2max": era5_dir.joinpath("t2max_era5.zarr"),
    "pr": era5_dir.joinpath("pr_era5.zarr"),
}

era5_ds = baeda.open_era5_dataset(era5_stores)

era5_extr = baeda.extract_era5_time_series(era5_ds, coords).load()

In [ ]:
# compute the indicators for the ERA5 data
for var_id in var_ids:
    era5_indicators = baeda.run_indicators(
        era5_extr[var_id].assign_coords(Method="ERA5").expand_dims(["Method"]),
        era5_extr[var_id],
    )
    era5_indicators.to_zarr(
        tmp_dir.joinpath(tmp_era5_discrete_idx_fp.format(var_id=var_id))
    )